In [76]:
import sys
sys.path.append('../../../src/')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle

from Model import Impf_DNAMLP
from torch.nn.functional import softmax

In [77]:
fold = '3.4'
outer_fold = f"{fold.split('.')[0]}.0"
group = 'Sella'
alg = 'RF'

In [78]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']
OUTER_FOLD_CSV_FILE = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
FOLD_CSV_FILE = os.path.join(TEST_CSV_DIR, group, f'{fold}_test.csv')
device = impf_cfg['device']
device

'cpu'

In [79]:
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)
print(len(impf))

2049


In [80]:
df = pd.read_csv(FOLD_CSV_FILE, index_col=0)
df_test = pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0)
df.shape, df_test.shape

((42, 10002), (59, 10002))

In [81]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0).iloc[:, -1]).value_counts()

Sella      34
Control    25
Name: class_group, dtype: int64

## Load random test sample and predict

In [82]:
rand = np.random.randint(df_test.shape[0])
sample = df_test.iloc[rand, :]
sample

cg22054918     0.003023
cg16476975     0.015417
cg25570913     0.052471
cg03940848     0.218212
cg02809746     0.010131
                 ...   
cg22513691     0.885517
cg00743629     0.887552
cg13621317     0.571651
label          CPH, ADM
class_group       Sella
Name: 9305651095_R06C02, Length: 10002, dtype: object

In [83]:
sample = df_test.iloc[rand,:].loc[impf]
sample

C:\Users\OA\AppData\Local\Temp/ipykernel_11476/1083929182.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  sample = df_test.iloc[rand,:].loc[impf]


cg00323305    0.100081
cg13392078      0.7397
cg10018522    0.210633
cg01181817    0.073812
cg25403488    0.104597
                ...   
cg18366169    0.087431
cg06998965    0.105729
cg18391978    0.983084
cg25072949    0.884488
cg12974388     0.01833
Name: 9305651095_R06C02, Length: 2049, dtype: object

In [84]:
sample_features = np.expand_dims(np.array(sample).astype(float), axis = 0)
# sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

In [85]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{fold}.pth')

n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE, map_location = torch.device('cpu')))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=2049, out_features=2, bias=True)
  )
)

In [86]:
sample_features.shape

(1, 2049)

In [87]:
logits = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
logits

tensor([[0.3258, 0.6742]], grad_fn=<SoftmaxBackward0>)